In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time

## NASA Mars News

In [2]:
#request response from the mars news webpage
news_url='https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
news_response=requests.get(news_url)

In [3]:
#create a BeautifulSoup object and define the parser
news_soup=BeautifulSoup(news_response.text,'html.parser')

In [4]:
title_div=news_soup.find('div',class_='content_title')
news_title=title_div.a.text.strip()
news_title

"NASA Readies Perseverance Mars Rover's Earthly Twin"

In [5]:
news_div=news_soup.find('div',class_='rollover_description_inner')
news_p=news_div.text.strip()
news_p

"Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."

## JPL Mars Space Images - Featured Image

### Mac Users

In [6]:
def init_browser():def
    # https://splinter.readthedocs.io/en/latest/drivers/chrome.html
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser('chrome', **executable_path, headless=False)

### Windows Users

In [7]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [8]:
browser=init_browser()
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
#brower will popup and visit the url
browser.visit(jpl_url)
time.sleep(5)

In [9]:
#click "FULL IMAGE" to get the full image of the featured image
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')

/Users/xiaoyuezhu/anaconda3/envs/pythonData/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [10]:
#now into 'FULL IMAGE'
full_html=browser.html
jpl_soup=BeautifulSoup(full_html,'html.parser')
img_figure=jpl_soup.find('figure',class_='lede')
img_tag=img_figure.a.img

In [11]:
img_tag

<img alt="Unusual arc-shaped, reddish streaks cut across the surface of Saturn's ice-rich moon Tethys in this enhanced-color mosaic from NASA's Cassini spacecraft." class="main_image" src="/spaceimages/images/largesize/PIA19637_hires.jpg" title="Unusual arc-shaped, reddish streaks cut across the surface of Saturn's ice-rich moon Tethys in this enhanced-color mosaic from NASA's Cassini spacecraft."/>

In [12]:
#will error: 'NoneType' object is not subscriptable when "RUN ALL" cells
#but if you manually rerun 2 cells above, error will be gone and able to extract the value

img_base_url='https://www.jpl.nasa.gov'
featured_img_url=img_base_url+img_tag['src']
browser.quit()

In [13]:
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19637_hires.jpg'

## Mars Weather

In [14]:
#open a new broswer
browser=init_browser()
weather_url='https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

# HTML Object.
weather_html = browser.html

# Parse HTML with Beautiful Soup
weather_soup = BeautifulSoup(weather_html, "html.parser")

In [15]:
tweet= weather_soup.find('div', attrs={'data-testid': 'tweet'})
spans=tweet.find_all('span')
weather_content=spans[4].text
browser.quit()

In [16]:
weather_content

'InSight sol 611 (2020-08-15) low -93.8ºC (-136.9ºF) high -15.9ºC (3.4ºF)\nwinds from the WNW at 7.3 m/s (16.3 mph) gusting to 17.9 m/s (40.2 mph)\npressure at 7.90 hPa'

## Mars Facts

In [28]:
fact_url='https://space-facts.com/mars/'
tables = pd.read_html(fact_url)

In [29]:
#the 1st table on the website is the one we want
fact_df=tables[0]

In [32]:
#rename columns
fact_df.columns = ['Description','Value']

#reindex the dataframe
fact_df.set_index('Description', inplace=True)

#convert the datafram to html, and export to a file
html_codes=fact_df.to_html()

In [34]:
fact_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [21]:
#open a new broswer
browser=init_browser()
hem_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)

# HTML Object.
hem_html = browser.html

# Parse HTML with Beautiful Soup
hem_soup = BeautifulSoup(hem_html, "html.parser")

In [22]:
image_links=hem_soup.find_all('div',class_='item')
image_links_list=[link.a['href'] for link in image_links]

In [23]:
image_links_list

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [24]:
hemisphere_image_urls=[]
base_image_url='https://astrogeology.usgs.gov'
for x in range(0,4):
    browser.visit(base_image_url+image_links_list[x])
    #give 2 seconds to fully load the page
    time.sleep(3)

    single_img_html = browser.html
    single_img_soup=BeautifulSoup(single_img_html,'html.parser')
    wide_img=single_img_soup.find('img',class_='wide-image')
    title=single_img_soup.find('h2',class_='title')

    
    hemisphere_image_urls.append({'title':title.text.replace(' Enhanced',''),
                                  'image_url':base_image_url+wide_img['src']})
    
    
    print(f'Done with the {x+1}th image')

browser.quit()

Done with the 1th image
Done with the 2th image
Done with the 3th image
Done with the 4th image


In [25]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]